Training Pipeline - YOLOv8 OBB - PSM Project

#Adicionar uma descrição

Step 1 - Downloading the dataset & setting up the folder

In [ ]:
pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mTwL8ga72R2LP9j2TkRt")
project = rf.workspace("pmec").project("rooftop-segmentation-nbosn")
version = project.version(2)
dataset = version.download("yolov8-obb")
                

In [ ]:
import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

Training the dataset

In [ ]:
pip install "ultralytics<=8.3.40"

In [ ]:
from ultralytics import YOLO

model = YOLO(“yolov8n-obb.pt”) # Loads a pre-trained model

results = model.train(data=f”{dataset.location}/yolov8-obb.yaml', epochs=100, imgsz=640)

Test the model

In [ ]:
pip install supervision

In [ ]:
model = YOLO('runs/obb/train2/weights/best.pt')

import os
import random

random_file = random.choice(os.listdir(f"{dataset.location}/test/images"))
file_name = os.path.join(f"{dataset.location}/test/images", random_file)

results = model(file_name)

import supervision as sv
import cv2

detections = sv.Detections.from_ultralytics(results[0])

oriented_box_annotator = sv.OrientedBoxAnnotator()
annotated_frame = oriented_box_annotator.annotate(
    scene=cv2.imread(file_name),
    detections=detections
)

sv.plot_image(image=annotated_frame, size=(16, 16))